In [1]:
# Dependencies

from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import pymongo

In [2]:
def init_browser():
    # Replacing the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)


In [3]:
# NASA news URL
browser = init_browser()
url = "https://mars.nasa.gov/news/" 
browser.visit(url)

In [5]:
# Scrapping url page into Beautifulsoup

new_html = browser.html
bsoup = bs(new_html, 'html.parser')

In [6]:
# Getting most recent article, title and date

# Find the latest Mars news.

#article = bsoup.find("div", class_="list_text").text

news_p = bsoup.find('div', class_='rollover_description_inner').text
news_title = bsoup.find("div", class_="content_title").text
news_date = bsoup.find("div", class_="list_date").text



#print(article)

print(news_date)
print(news_title)
print(news_p)

February 28, 2019
After a Reset, Curiosity Is Operating Normally
Curiosity has returned to science operations and is once again exploring the clay unit. 


In [7]:
# Accessing the JPL Mars URL

url2 = "https://jpl.nasa.gov/spaceimages/?search=&category=Mars" 
browser = init_browser()
browser.visit(url2)

In [8]:
# Scraping the browser into soup and using soup to find Mars image
# Saving image url to a variable called `img_url`
img_url ='0'
new_html = browser.html
bsoup = bs(new_html, 'html.parser')
image = bsoup.find('img', class_="thumb")["src"]
img_url = "https://jpl.nasa.gov"+image
featured_image_url = img_url
browser.quit()

In [9]:
# Requesting library to download/save/display image from the above img_url

import requests
import shutil

response = requests.get(featured_image_url, stream=True)
with open('img.jpg', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)

In [10]:
# Displaying image with IPython.display

from IPython.display import Image
Image(url='img.jpg')

In [15]:
# Visiting Mars Weather twitter/scraping the latest tweet.

twt_url = 'https://twitter.com/marswxreport?lang=en'
browser = init_browser()
browser.visit(twt_url)

new_html = browser.html
bsoup = bs(new_html, 'html.parser')
twt_rslt = bsoup.find_all('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
# twt_rslt = twt_rslt.replace('pic.twitter.com/1msjBvhiu7','')

# Acessing next tweet from the twitter url

twt_weather = twt_rslt[1].text
twt_weather_twt = twt_weather.replace('pic.twitter.com/WlR4gr8gpC','')


# Adding the weather data 

mars_weather = twt_weather_twt

print(twt_weather_twt)
browser.quit()


InSight sol 90 (2019-02-26) low -93.9ºC (-137.0ºF) high -12.1ºC (10.2ºF)
winds from the SW at 4.2 m/s (9.3 mph) gusting to 12.0 m/s (26.9 mph)
pressure at 7.20 hPa


In [16]:
# Scraping table data Mars web page into Pandas

url3 = "http://space-facts.com/mars/"
browser =init_browser()
browser.visit(url3)

In [17]:
# Populating data into a dataframe, cleaning and creating into an HTML table

import pandas as pd
grab = pd.read_html(url3)
fact_df = pd.DataFrame(grab[0])
fact_df = fact_df.rename(columns={0:'Description',1: 'Values'})
fact_df = fact_df.set_index('Description')
df_to_html = fact_df.to_html(classes='mars_scrape_info')
df_to_html = df_to_html.replace("\n"," ")
fact_df


,Values
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [20]:
# Scraping pictures of the hemispheres from USGS Astrogeology web page

url4 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1 =target&v1=Mars"
browser =init_browser()
browser.visit(url4)

In [21]:
# Loading 4 images and load them into dictionary

import time

new_html = browser.html
bsoup = bs(new_html, 'html.parser')
    
image_mars = []

for item in range(4):    
    images = browser.find_by_tag('h3')
    images[item].click()
    new_html = browser.html
    bsoup = bs(new_html, 'html.parser')
    newContent = bsoup.find("img", class_="wide-image")["src"]
    img_title = bsoup.find("h2", class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ newContent
    img_dict = {"title":img_title,"img_url":img_url}
    image_mars.append(img_dict)
    time.sleep(1)
    browser.back()



In [22]:
image_mars

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [23]:
browser.quit()